# The Price is Right - Fixed Version

This notebook fixes the issue where existing deals disappear from the table when the system makes new calls.

**Key Fix**: The table now continuously shows current memory during updates, so existing deals remain visible while new ones are being searched.


In [1]:
# Imports
import sys
import os

# Change working directory to week8 where deal_agent_framework expects to run
# This ensures all relative paths (models, database) work correctly
notebook_dir = os.getcwd()
week8_dir = os.path.join(notebook_dir, '..', '..')
os.chdir(week8_dir)
print(f"Working directory: {os.getcwd()}")

import logging
import queue
import threading
import time
import gradio as gr
from deal_agent_framework import DealAgentFramework
from agents.deals import Opportunity, Deal
from log_utils import reformat
import plotly.graph_objects as go


Working directory: c:\Users\hp\projects\gen-ai\llm_engineering\week8


In [2]:
# Helper Functions

class QueueHandler(logging.Handler):
    def __init__(self, log_queue):
        super().__init__()
        self.log_queue = log_queue

    def emit(self, record):
        self.log_queue.put(self.format(record))


def html_for(log_data):
    """Convert log data to HTML format for display"""
    output = '<br>'.join(log_data[-18:])
    return f"""
    <div id="scrollContent" style="height: 400px; overflow-y: auto; border: 1px solid #ccc; background-color: #222229; padding: 10px;">
    {output}
    </div>
    """


def setup_logging(log_queue):
    """Setup logging to capture messages in a queue"""
    handler = QueueHandler(log_queue)
    formatter = logging.Formatter(
        "[%(asctime)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S %z",
    )
    handler.setFormatter(formatter)
    logger = logging.getLogger()
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)


def get_plot():
    """Generate 3D visualization of vector database - handles empty database gracefully"""
    try:
        documents, vectors, colors = DealAgentFramework.get_plot_data(max_datapoints=1000)
        
        # Check if we have any data
        if len(vectors) == 0:
            # Return placeholder plot if database is empty
            fig = go.Figure()
            fig.update_layout(
                title='Vector Database Empty',
                height=400,
                annotations=[dict(
                    text="The vector database is empty.<br>Run the data loading notebook (day2.0) to populate it.",
                    x=0.5,
                    y=0.5,
                    xref="paper",
                    yref="paper",
                    showarrow=False,
                    font=dict(size=14)
                )]
            )
            return fig
        
        # Normal case: create 3D scatter plot
        fig = go.Figure(data=[go.Scatter3d(
            x=vectors[:, 0],
            y=vectors[:, 1],
            z=vectors[:, 2],
            mode='markers',
            marker=dict(size=2, color=colors, opacity=0.7),
        )])
        
        fig.update_layout(
            scene=dict(xaxis_title='x', 
                       yaxis_title='y', 
                       zaxis_title='z',
                       aspectmode='manual',
                       aspectratio=dict(x=2.2, y=2.2, z=1),
                       camera=dict(
                           eye=dict(x=1.6, y=1.6, z=0.8)
                       )),
            height=400,
            margin=dict(r=5, b=1, l=5, t=2)
        )
        return fig
    except Exception as e:
        # Handle any errors gracefully
        fig = go.Figure()
        fig.update_layout(
            title='Error Loading Vector Database',
            height=400,
            annotations=[dict(
                text=f"Error: {str(e)}<br><br>Make sure the vector database is set up correctly.<br>Run day2.0 notebook to populate it.",
                x=0.5,
                y=0.5,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=12)
            )]
        )
        return fig


def create_opportunity_from_dict(data: dict) -> Opportunity:
    """Helper function to create Opportunity from dictionary - uses Deal and Opportunity classes"""
    deal = Deal(**data['deal']) if isinstance(data['deal'], dict) else data['deal']
    return Opportunity(deal=deal, estimate=data['estimate'], discount=data['discount'])


def validate_opportunities(opportunities) -> list:
    """Validate and ensure all items are Opportunity instances - uses Opportunity class"""
    validated = []
    for opp in opportunities:
        if not isinstance(opp, Opportunity):
            if isinstance(opp, dict):
                opp = create_opportunity_from_dict(opp)
            else:
                continue
        validated.append(opp)
    return validated


In [ ]:
# Main App Class

class App:

    def __init__(self):    
        self.agent_framework = None

    def get_agent_framework(self):
        """Get or initialize the agent framework"""
        if not self.agent_framework:
            self.agent_framework = DealAgentFramework()
            self.agent_framework.init_agents_as_needed()
        return self.agent_framework

    def table_for(self, opps):
        """Convert opportunities to table format - uses Opportunity and Deal classes"""
        # Validate opportunities are Opportunity instances
        validated_opps = validate_opportunities(opps)
        return [[opp.deal.product_description, f"${opp.deal.price:.2f}", f"${opp.estimate:.2f}", f"${opp.discount:.2f}", opp.deal.url] 
                for opp in validated_opps 
                if isinstance(opp, Opportunity)]

    def update_output(self, log_data, log_queue, result_queue):
        """Keep showing current memory during updates - fixes disappearing table issue"""
        framework = self.get_agent_framework()
        current_table = self.table_for(framework.memory)
        
        while True:
            try:
                message = log_queue.get_nowait()
                log_data.append(reformat(message))
                # Always refresh table from current memory during updates
                current_table = self.table_for(framework.memory)
                yield log_data, html_for(log_data), current_table
            except queue.Empty:
                try:
                    # When result is ready, update with final result
                    final_result = result_queue.get_nowait()
                    yield log_data, html_for(log_data), final_result
                    return
                except queue.Empty:
                    # Continue showing current memory while waiting
                    current_table = self.table_for(framework.memory)
                    yield log_data, html_for(log_data), current_table
                    time.sleep(0.1)

    def do_run(self):
        """Run framework and return updated table"""
        import datetime
        framework = self.get_agent_framework()
        # Log to the Gradio display (will show up in logs panel)
        logging.info(f"⏰ TIMER TRIGGERED at {datetime.datetime.now().strftime('%H:%M:%S')} - Current memory: {len(framework.memory)} deals")
        new_opportunities = framework.run()
        logging.info(f"✅ Scan complete - Total deals in memory: {len(framework.memory)}")
        return self.table_for(new_opportunities)

    def run_with_logging(self, initial_log_data):
        """Run agent framework with logging in a separate thread"""
        log_queue = queue.Queue()
        result_queue = queue.Queue()
        setup_logging(log_queue)
        
        def worker():
            result = self.do_run()
            result_queue.put(result)
        
        thread = threading.Thread(target=worker)
        thread.start()
        
        for log_data, output, final_result in self.update_output(initial_log_data, log_queue, result_queue):
            yield log_data, output, final_result

    def do_select(self, selected_index: gr.SelectData):
        """Handle deal selection - send alert"""
        framework = self.get_agent_framework()
        opportunities = framework.memory
        row = selected_index.index[0]
        if row < len(opportunities):
            opportunity = opportunities[row]
            framework.planner.messenger.alert(opportunity)
            return f"Alert sent for: {opportunity.deal.product_description[:50]}..."
        return "No opportunity found at that index"

    def load_initial(self):
        """Load initial state with existing deals - uses Opportunity and Deal classes"""
        framework = self.get_agent_framework()
        # Ensure memory contains Opportunity instances
        opportunities = validate_opportunities(framework.memory)
        initial_table = self.table_for(opportunities)
        return [], "", initial_table

    def run(self):
        """Launch the Gradio interface"""
        with gr.Blocks(title="The Price is Right", fill_width=True) as ui:
            
            log_data = gr.State([])
    
            with gr.Row():
                gr.Markdown('<div style="text-align: center;font-size:24px"><strong>The Price is Right</strong> - Autonomous Agent Framework that hunts for deals</div>')
            with gr.Row():
                gr.Markdown('<div style="text-align: center;font-size:14px">A proprietary fine-tuned LLM deployed on Modal and a RAG pipeline with a frontier model collaborate to send push notifications with great online deals.</div>')
            with gr.Row():
                opportunities_dataframe = gr.Dataframe(
                    headers=["Deals found so far", "Price", "Estimate", "Discount", "URL"],
                    wrap=True,
                    column_widths=[6, 1, 1, 1, 3],
                    row_count=10,
                    col_count=5,
                    max_height=400,
                )
            with gr.Row():
                with gr.Column(scale=1):
                    logs = gr.HTML()
                with gr.Column(scale=1):
                    plot = gr.Plot(value=get_plot(), show_label=False)
        
            # Initial load - show existing deals
            ui.load(self.load_initial, inputs=[], outputs=[log_data, logs, opportunities_dataframe])

            # Timer that runs every 5 minutes (300 seconds)
            timer = gr.Timer(value=10, active=True)
            timer.tick(self.run_with_logging, inputs=[log_data], outputs=[log_data, logs, opportunities_dataframe])

            # Selection handler
            selection_feedback = gr.Textbox(visible=False)
            opportunities_dataframe.select(self.do_select, inputs=[], outputs=[selection_feedback])
        
        ui.launch(share=False, inbrowser=True)


In [ ]:
# Run the application
app = App()
app.run()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


[2025-10-30 20:52:39 +0100] [Agents] [INFO] [Agent Framework] Initializing Agent Framework
[2025-10-30 20:52:39 +0100] [Agents] [INFO] [Planning Agent] Planning Agent is initializing
[2025-10-30 20:52:39 +0100] [Agents] [INFO] [Scanner Agent] Scanner Agent is initializing
[2025-10-30 20:52:39 +0100] [Agents] [INFO] [Scanner Agent] Scanner Agent is ready
[2025-10-30 20:52:39 +0100] [Agents] [INFO] [Ensemble Agent] Initializing Ensemble Agent
[2025-10-30 20:52:39 +0100] [Agents] [INFO] [Specialist Agent] Specialist Agent is initializing - connecting to modal
[2025-10-30 20:52:39 +0100] [Agents] [INFO] [Specialist Agent] Specialist Agent is ready
[2025-10-30 20:52:39 +0100] [Agents] [INFO] [Frontier Agent] Initializing Frontier Agent
[2025-10-30 20:52:39 +0100] [Agents] [INFO] [Frontier Agent] Frontier Agent is set up with DeepSeek
[2025-10-30 20:52:39 +0100] [Agents] [INFO] Use pytorch device_name: cpu
[2025-10-30 20:52:39 +0100] [Agents] [INFO] Load pretrained SentenceTransformer: sente